In [13]:
import pandas as pd

In [14]:
urls_df = pd.read_excel('Input.xlsx')

In [15]:
urls_df.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [ ]:
import numpy as np
import math
from bs4 import BeautifulSoup
import requests
import os
from fake_useragent import UserAgent

# Web Scraping part

. Using BeautifulSoup to extract the article title and content data 
. Concatenating both the title and content and returning from the function
. New column 'ARTICLE' is created with the article data of all the URLs

In [ ]:
def url_text(url):
    paragraphs=[]
    data = requests.get(url,headers={"User-Agent": "XY"}).text
    soup = BeautifulSoup(data,'lxml')
    title=soup.find_all('h1')[0].text.strip()
    p=soup.find_all('p')
    for paragraph in p:
        paragraph=paragraph.text.strip()
        paragraphs.append(paragraph)
    text=''.join(paragraphs)
    text=title+" "+text
    return text

In [ ]:
urls_df['URL'][36]

In [ ]:
url_text(urls_df['URL'][36])

In [ ]:
url_text(urls_df['URL'][37])

In [ ]:
urls_df.drop(37, inplace=True)

In [ ]:
urls_df['ARTICLE'] = urls_df['URL'].apply(url_text)

In [ ]:
urls_df['ARTICLE']

Saving the extracted text into individual text files with URL_ID as filename

In [ ]:
for i in urls_df.index:
    filename=str(urls_df['URL_ID'][i])
    text=str(urls_df['ARTICLE'][i])
    path='articles/'+filename+'.txt'
    
    if not os.path.exists(path):
        file = open(path,'w',encoding='utf-8')
        file.write(text)
        file.close()

In [ ]:
from nltk.tokenize import RegexpTokenizer , sent_tokenize

# Loading positive and negative words as well as stopwords from text files

In [ ]:
with open('MasterDictionary/positive-words.txt','r') as file:
    positive_words = file.read().lower()
positive_words = positive_words.split('\n')
#positive_words

In [ ]:
with open('MasterDictionary/negative-words.txt','r', errors='ignore') as file:
    negative_words = file.read().lower()
negative_words = negative_words.split('\n')
#negative_words

In [ ]:
with open('StopWords/StopWords_Auditor.txt','r') as file:
    stopwords_auditor = file.read().lower()
stopwords_auditor = stopwords_auditor.split('\n')
stopwords_auditor

In [ ]:
with open('StopWords/StopWords_Currencies.txt','r', errors='ignore') as file:
    stopwords_currencies = file.read().lower()
stopwords_currencies = stopwords_currencies.split('\n')
stopwords_currencies

In [ ]:
stopwords_currency = []
for currency in stopwords_currencies:
    stopwords_currency.append(currency.split('|')[0].strip())
stopwords_currency

In [ ]:
with open('StopWords/StopWords_DatesandNumbers.txt','r') as file:
    stopwords_datesandnumbers = file.read().lower()
stopwords_datesandnumbers = stopwords_datesandnumbers.split('\n')
stopwords_datesandnumber = []
for date in stopwords_datesandnumbers:
    if "|" in date:
        stopwords_datesandnumber.append(date.split("|")[0].strip())
    else:
        stopwords_datesandnumber.append(date.strip())
stopwords_datesandnumber

In [ ]:
with open('StopWords/StopWords_Generic.txt','r') as file:
    stopwords_generic = file.read().lower()
stopwords_generic = stopwords_generic.split('\n')
stopwords_generic

In [ ]:
with open('StopWords/StopWords_GenericLong.txt','r') as file:
    stopwords_genericlong = file.read().lower()
stopwords_genericlong = stopwords_genericlong.split('\n')
stopwords_genericlong

In [ ]:
with open('StopWords/StopWords_Geographic.txt','r') as file:
    stopwords_geo = file.read().lower()
stopwords_geo = stopwords_geo.split('\n')
stopwords_geographic = []
for geo in stopwords_geo:
    if "|" in geo:
        stopwords_geographic.append(geo.split('|')[0].strip())
    else:
        stopwords_geographic.append(geo.strip())
stopwords_geographic

In [ ]:
with open('StopWords/StopWords_Names.txt','r') as file:
    stopwords_names = file.read().lower()
stopwords_names = stopwords_names.split('\n')
stopwords_name = []
for name in stopwords_names:
    if "|" in name:
        stopwords_name.append(name.split('|')[0].strip())
    else:
        stopwords_name.append(name.strip())
stopwords_name

In [ ]:
stop_words = []
stop_words = stopwords_auditor + stopwords_currency + stopwords_datesandnumber + stopwords_generic + stopwords_genericlong + stopwords_geographic + stopwords_name

In [ ]:
stop_words;

# Defining functions for calculating required variables after analysis

- RegexpTokenizer() is used to tokenize the article text, the stopwords in these tokens are filtered out. After filtering, the filtered tokens are then used to conduct the analysis on the text. 

In [ ]:
def tokenz(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = str(text)
    tokens = tokenizer.tokenize(text)
    filter = []
    for token in tokens:
        if token not in stop_words:
            filter.append(token)
    return filter

Positive Score: This score is calculated by assigning the value of +1 for each word if found in the Positive Dictionary and then adding up all the values.

In [ ]:
def positive_score(text):
    tokens = tokenz(text)
    pos_score = 0
    for token in tokens:
        if token in positive_words:
            pos_score += 1
    return pos_score

Negative Score: This score is calculated by assigning the value of -1 for each word if found in the Negative Dictionary and then adding up all the values. We multiply the score with -1 so that the score is a positive number.


In [ ]:
def negative_score(text):
    tokens = tokenz(text)
    neg_score = 0
    for token in tokens:
        if token in negative_words:
            neg_score -= 1
    return neg_score * -1

Polarity Score: This is the score that determines if a given text is positive or negative in nature. It is calculated by using the formula: 
Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
Range is from -1 to +1


In [ ]:
def polarity_score(pos_score,neg_score):
    polar_score = (pos_score - neg_score)/((pos_score + neg_score)+0.000001)
    return polar_score

Subjectivity Score: This is the score that determines if a given text is objective or subjective. It is calculated by using the formula: 
Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
Range is from 0 to +1


In [ ]:
def subjectivity_score(pos_score,neg_score,text):
    tokens = tokenz(text)
    sub_score = (pos_score + neg_score)/((len(tokens)) + 0.000001)
    return sub_score

Average Sentence Length = the number of words / the number of sentences


In [ ]:
def avg_sentence_length(text):
    words = len(tokenz(text))
    sentences = len(text.split('.'))
    avg_length = words / sentences
    return avg_length

Percentage of Complex words = the number of complex words / the number of words 

In [ ]:
def perc_complex(text):
    tokens = tokenz(text)
    complex_cnt = 0
    for token in tokens:
        vowels = 0
        if token.endswith(('es','ed')):
            pass
        else:
            for l in token:
                if (l == 'a' or l == 'e' or l == 'i' or l == 'o' or l == 'u'):
                    vowels += 1
            if vowels > 2:
                complex_cnt += 1
    if len(token) != 0:
        return complex_cnt/len(token)

Complex words are words in the text that contain more than two syllables.


In [ ]:
def complex_word_count(text):
    tokens = tokenz(text)
    complex_cnt = 0
    for token in tokens:
        vowels = 0
        if token.endswith(('es','ed')):
            pass
        else:
            for l in token:
                if (l == 'a' or l == 'e' or l == 'i' or l == 'o' or l == 'u'):
                    vowels += 1
            if vowels > 2:
                complex_cnt += 1
    return complex_cnt

Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)


In [ ]:
def fog_index(avg_sent_length,percentage_complex):
    return 0.4 * (avg_sent_length + percentage_complex)    

We count the total cleaned words present in the text by 
removing the stop words 
removing any punctuations like ? ! , . from the word before counting.


In [ ]:
def word_count(text):
    cnt = len(tokenz(text))
    return cnt

We count the number of Syllables in each word of the text by counting the vowels present in each word. We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.


In [ ]:
def syllable_count_word(text):
    words = tokenz(text)
    words_cnt = len(words)
    vowels = 0
    for word in words:
        if word.endswith(('es','ed')):
            pass
        else:
            for l in word:
                if (l == 'a' or l == 'e' or l == 'i' or l == 'o' or l == 'u'):
                    vowels += 1
    return vowels / words_cnt    

In [ ]:
import re

To calculate Personal Pronouns mentioned in the text, we use regex to find the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken so that the country name US is not included in the list.


In [ ]:
def cal_personal_pronouns(text):
    words = tokenz(text)
    pronoun_re = r'\b(I|my|we|us|ours)\b'
    matches = re.findall(pronoun_re,text)
    return len(matches)

Average Word Length is calculated by the formula:
Sum of the total number of characters in each word/Total number of words


In [ ]:
def avg_word_length(text):
    words = tokenz(text)
    charcnt = 0
    for word in words:
        charcnt += len(word.strip())
    if len(words) != 0:
        return charcnt / len(words)

# Applying all the functions to the dataframe for respective new columns for the calculated variables

- for functions with inputs from more than one columns in the dataframe, np.vectorize() is used. For the rest pd.apply() is used.


In [ ]:
urls_df['POSITIVE SCORE'] = urls_df['ARTICLE'].apply(positive_score)

In [ ]:
urls_df['NEGATIVE SCORE'] = urls_df['ARTICLE'].apply(negative_score)

In [ ]:
urls_df['POLARITY SCORE'] = np.vectorize(polarity_score)(urls_df['POSITIVE SCORE'],urls_df['NEGATIVE SCORE'])

In [ ]:
urls_df['SUBJECTIVITY SCORE'] = np.vectorize(subjectivity_score)(urls_df['POSITIVE SCORE'],urls_df['NEGATIVE SCORE'],urls_df['ARTICLE'])

In [ ]:
urls_df['AVG SENTENCE LENGTH'] = urls_df['ARTICLE'].apply(avg_sentence_length)

##### 

In [ ]:
urls_df['PERCENTAGE OF COMPLEX WORDS'] = urls_df['ARTICLE'].apply(perc_complex)

In [ ]:
urls_df['FOG INDEX'] = np.vectorize(fog_index)(urls_df['AVG SENTENCE LENGTH'],urls_df['PERCENTAGE OF COMPLEX WORDS'])

In [ ]:
urls_df['AVG NUMBER OF WORDS PER SENTENCE'] = urls_df['ARTICLE'].apply(avg_word_length)

In [ ]:
urls_df['COMPLEX WORD COUNT'] = urls_df['ARTICLE'].apply(complex_word_count)

In [ ]:
urls_df['WORD COUNT'] = urls_df['ARTICLE'].apply(word_count)

In [ ]:
urls_df['SYLLABLE PER WORD'] = urls_df['ARTICLE'].apply(syllable_count_word)

In [ ]:
urls_df['PERSONAL PRONOUNS'] = urls_df['ARTICLE'].apply(cal_personal_pronouns)

In [ ]:
urls_df['AVG WORD LENGTH'] = urls_df['ARTICLE'].apply(avg_word_length)

In [ ]:
urls_df.head(12)

In [ ]:
urls_df.drop('ARTICLE',axis=1,inplace=True)

# Loading all the calculated variables from the dataframe into the given output excel file in the same structure

In [ ]:
urls_df.to_excel("Output Data Structure.xlsx",encoding='utf-8')